# Adversairal training
Exercise shows how to increase robustness of model by adversarial training.

In [1]:
from utils import load_news20
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from wildnlp.aspects import *
from wildnlp.aspects.utils import compose
import pandas as pd

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


# Download data

In [2]:
# download the 20 News Group dataset, this may take a minute
train_data, test_data, class_names = load_news20()

# extract train and test datasets
X_raw_train, y_train = train_data
X_raw_test, y_test = test_data

In [3]:
class_names

['atheism',
 'graphics',
 'ms-windows.misc',
 'pc.hardware',
 'mac.hardware',
 'ms-windows.x',
 'misc.forsale',
 'autos',
 'motorcycles',
 'baseball',
 'hockey',
 'crypt',
 'electronics',
 'med',
 'space',
 'christian',
 'guns',
 'mideast',
 'politics.misc',
 'religion.misc']

# Corrupt train data set

In [4]:
# Create a composed corruptor function.
# Functions will be applied in the same order they appear.
composed = compose(QWERTY(words_percentage=30), Swap(transform_percentage=30), RemoveChar(words_percentage=30))
X_raw_train_corrupt = [composed(x) for x in X_raw_train]

Concatenate clear and corrupt data

In [5]:
X_raw_train_new =  X_raw_train + X_raw_train_corrupt
y_train_new = np.concatenate((y_train, y_train))

# Corrupt test data set

In [6]:
# Create a composed corruptor function.
# Functions will be applied in the same order they appear.
composed = compose(QWERTY(words_percentage=30), Swap(transform_percentage=30), RemoveChar(words_percentage=30))
X_raw_test_corrupt = [composed(x) for x in X_raw_test]

In [7]:
example_index = 1

Clear

In [8]:
X_raw_test[example_index]

'From: Rick Miller <rick@ee.uwm.edu>\nSubject: X-Face?\nOrganization: Just me.\nLines: 17\nDistribution: world\nNNTP-Posting-Host: 129.89.2.33\nSummary: Go ahead... swamp me.  <EEP!>\n\nI\'m not familiar at all with the format of these "X-Face:" thingies, but\nafter seeing them in some folks\' headers, I\'ve *got* to *see* them (and\nmaybe make one of my own)!\n\nI\'ve got "dpg-view" on my Linux box (which displays "uncompressed X-Faces")\nand I\'ve managed to compile [un]compface too... but now that I\'m *looking*\nfor them, I can\'t seem to find any X-Face:\'s in anyones news headers!  :-(\n\nCould you, would you, please send me your "X-Face:" header?\n\nI *know* I\'ll probably get a little swamped, but I can handle it.\n\n\t...I hope.\n\nRick Miller  <rick@ee.uwm.edu> | <ricxjo@discus.mil.wi.us>   Ricxjo Muelisto\nSend a postcard, get one back! | Enposxtigu bildkarton kaj vi ricevos alion!\n          RICK MILLER // 16203 WOODS // MUSKEGO, WIS. 53150 // USA\n'

Corrupt

In [9]:
X_raw_test_corrupt[example_index]

'Fo5m: Rick Millre < rick @ ee. ume. edu > Suvejct: X-Face? Organizatoin: Just m3. Lines: 17 Disrinution: word NNTP-Positng-Host: 192. 89. . 33 Sumamry: Go aeda. .. swmap e. < EEP! > Im\' nt fsmiliar t al wihh teh fomrat 0f thsee" XF-ace: " tingeis, but after seein them i xoem fokle\' haeders, I\'ve *got* go *see* thmr (adn maybe maek on og my own)! I\'ve gto" dpg-view" n y Linu bxo (hwich dislpays" uncompressd X-Fzecs" ) afn I\'be managed o cmopile [ un ] compfacd oi. .. btu nwo th6a I\'m *looikng* fro thme, I can\'t serm to find aym XF-ace: \'s in anones nes yeaders! : -( Coud io, wolud yku, pleae sner n yor" X-Face: " heaer? I *knwo* Il\'l probbaly get litt. swamepd, bu I na hanle it. .. . I npo. Ric Milelr < dcik @ ed. uwm. eud > | < rixjxo @ dicsud. mil. wi. uw > Ricxjo Meuliso Sesn z psotxard, yte oen bcak! | Snposxitgu bildarton aj vi riecvos aino! ICK MILLER / / 12603 WO9DS / / MUSLEOG, WIS. 53150 / / USA'

# Prepare clear and corrupt data

In [10]:
# maximum vocabulary size 
max_features = 20000

# vectorize the data using tfidf, this time we'll use the whole dataset
tfidf = TfidfVectorizer(max_features=max_features, stop_words='english')
X_train = tfidf.fit_transform(np.array(X_raw_train_new))
X_test = tfidf.transform(X_raw_test)
X_test_corrupt = tfidf.transform(X_raw_test_corrupt)

MemoryError: 

# Prepare and test model

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline

# initialize and train a naive bayes model
model = MultinomialNB(alpha=1)
model.fit(X=X_train, y=y_train_new)

Evaluate its performance on the clear test set

In [ ]:
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

Evaluate its performance on the corrupt test set

In [ ]:
y_pred_corrupt = model.predict(X_test_corrupt)
accuracy_score(y_test, y_pred_corrupt)

In [ ]:
# we'll make a simple pipeline to fallback to raw text
pipe = make_pipeline(tfidf, model)

# Black-box explainer

In [ ]:
from utils import most_important_words_black

In [ ]:
# feel free to change to any of the above indices
idx_to_explain = 21
x = X_raw_test_corrupt[idx_to_explain]
y = y_test[idx_to_explain]
x

In [ ]:
class_names[y]

In [ ]:
%%time
words_imp = most_important_words_black(x, y, pipe.predict_proba)

In [ ]:
for ind, word, imp in words_imp:
    print(ind, word, imp)

# Lime

In [ ]:
from lime.lime_text import LimeTextExplainer


explainer = LimeTextExplainer(class_names=class_names)

In [ ]:
%%time
exp = explainer.explain_instance(x, pipe.predict_proba,  num_features=10, top_labels=1)

In [ ]:
exp.show_in_notebook()

# Geval

In [ ]:
np.savetxt("geval_adv_train/out.tsv",y_pred, fmt='%i')

In [ ]:
! geval -t  geval_adv_train --tokenizer 13a  --metric Accuracy --worst-features > worst_features_adv_train.txt